# BLAS and LAPACK

We've seen a bit of dense linear algebra using numpy and scipy.  Now we're going to look under the hood.

Regardless of what language you're using, chances are if you're doing numerical linear algebra, you are able to take advantage of libraries of code which implement most common linear algebra routines and factorizations.

[Basic Linear Algebra Subprograms (BLAS)](https://www.netlib.org/blas/)

[Linear Algebra PACKage (LAPACK)](https://www.netlib.org/lapack/)

These libraries are wrapped by scipy, which exposes an interface.

### Why should you care?

First, you probably shouldn't be writing your own basic linear algebra routines.
1.  It takes time to write them
2. Even if you know what you're doing, there's a chance you have a bug
3. Performance optimization is involved

It is entirely possible to do linear algebra in Python without ever worrying about the libraries under the hood.  However, maybe you are prototyping an algorithm in Python, and then want to write optimized code in C/fortran.  In this case, it is good to be able to translate what you're doing into BLAS/LAPACK routines.



## BLAS Routines

[scipy BLAS interface](https://docs.scipy.org/doc/scipy/reference/linalg.blas.html)

BLAS implements *basic* linear algebra routines like dot product, matrix-vector product, and matrix-matrix product as well as triangular solves.

In [1]:
%pylab inline
from scipy.linalg import blas

Populating the interactive namespace from numpy and matplotlib


In [2]:
n = 1000
A = np.array(np.random.randn(n,n), order='F')
B = np.array(np.random.randn(n,n), order='F')


C_np = A @ B # numpy
C_blas = blas.dgemm(1.0, A, B) # BLAS
np.linalg.norm(C_np - C_blas)

0.0

In [3]:
%timeit C_np = A @ B # numpy

16.9 ms ± 1 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%timeit C_blas = blas.dgemm(1.0, A, B) # blas

19.2 ms ± 2.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### BLAS Naming Conventions

BLAS functions have short names (like `dgemm`) that look a little cryptic at first.  There is a pattern to the names though.  See [here](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-fortran/top/blas-and-sparse-blas-routines/blas-routines/naming-conventions-for-blas-routines.html) for reference.

```
blas.<character><name><mod>
```

The `<character>` field can be
1. `s`: single precision float
2. `d`: double precision float
3. `c`: single precision complex float
4. `z`: double precision complex float

The `<name>` field indicate either the operation type, or matrix type.

The `<mod>` field indicates additional details about the operation.

For example, `dgemm = d + ge + mm` has `d` as the `<character>`, `ge` for `<name>` (general matrix), and `mm` for `<mod>` (matrix multiplication).

### BLAS Levels

BLAS is split up into 3 conceptual levels
1. Level 1 - vector-vector operations
2. Level 2 - matrix-vector operations
3. Level 3 - matrix-matrix operations

Level 3 operations are most efficient, since they can take the most advantage of memory performance optimizations.

#### BLAS Level 1

Some vector-vector operations (insert the appropriate `<character>`)
1. [`axpy`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.blas.daxpy.html) ($a x + y$)
2. [`dot`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.blas.ddot.html) (dot product $x^H x$)
3. [`nrm2`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.blas.dnrm2.html) ($\|x\|_2$)

In [5]:
n = 1000
x = np.random.rand(n)
y = np.random.rand(n)

z = blas.daxpy(x, y, a=1.0)
xx = blas.ddot(x, x)
x2 = blas.dnrm2(x)
np.sqrt(xx) - x2

0.0

#### BLAS Level 2

Some matrix-vector operations (insert appropriate `<character>`)

#### BLAS Level 3

Some matrix-matrix operations (insert appropriate `<character>`)

## LAPACK Routines

[scipy LAPACK interface](https://docs.scipy.org/doc/scipy/reference/linalg.lapack.html)